<a href="https://colab.research.google.com/github/nabajitdey/iDIL_image_processing/blob/master/Train_YoloV3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [3]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Wed Jun 24 21:20:12 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

**1) Clone the Darknet**



In [5]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 13738, done.
remote: Total 13738 (delta 0), reused 0 (delta 0), pack-reused 13738
Receiving objects: 100% (13738/13738), 12.31 MiB | 6.94 MiB/s, done.
Resolving deltas: 100% (9372/9372), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

**3) Configure Darknet network for training YOLO V3**

In [7]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [8]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [9]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [10]:
!echo "pool" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

In [14]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/grand-hotel-de-la-ville-garden-swimming-pool.jpg', 'data/obj/w-hotel-swimming-pool-hong-kong.jpg', 'data/obj/swimming-pools-250x250.jpg', 'data/obj/Asian-Paints-Swimming-Pool-Oberoi-Udaivilas.jpg', 'data/obj/images.jpg', 'data/obj/images (8).jpg', 'data/obj/home-design.jpg', 'data/obj/dam-images-travel-2015-hotel-pools-best-hotel-pools-01.jpg', 'data/obj/contemporary-swimming-pool-and-hot-tub.jpg', 'data/obj/images (5).jpg', 'data/obj/images (12).jpg', 'data/obj/udaipur.jpg', 'data/obj/10. Skypool - Shangri-La Hotel, At The Shard, London.jpg', 'data/obj/photo2jpg.jpg', 'data/obj/jaypee-residency-manor.jpg', 'data/obj/images (2).jpg', 'data/obj/pool--v17337198.jpg', 'data/obj/pool.jpg', 'data/obj/images (11).jpg', 'data/obj/17673-1-hotel_carousel_large.jpg', 'data/obj/the-pool (1).jpg', 'data/obj/140320134903-hotel-indoor-pools-w-verbier-horizontal-large-gallery.jpg', 'data/obj/Infinity_pool_featured_image.jpg', 'data/obj/6019ab2a21f8f86c7f51e179a8767cf7.jpg', 'data/obj/18169

In [15]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000444, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000537, iou_loss = 0.000000, total_loss = 0.000537 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000206, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000520, iou_loss = 0.000000, total_loss = 0.000520 
 total_bbox = 28231, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.660474, GIOU: 0.630247), Class: 0.900776, Obj: 0.040282, No Obj: 0.002607, .5R: 1.000000, .75R: 0.000000, count: 4, class_loss = 0.962830, iou_loss = 0.430509, total_loss = 1.393339 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 